In [1]:
import json
import os

import openai
from papermage import Document

from papermage_components.hf_token_classification_predictor import HfTokenClassificationPredictor

In [2]:
json_filename = "data/Midyear_Review_Papers_Parsed/On the creep performance of the Ti‐6Al‐4V alloy processed by additive manufacturing.json"

with open(json_filename) as f:
  parsed_paper = Document.from_json(json.load(f))

In [3]:
parsed_paper

Document with 27 layers: ['symbols', 'images', 'metadata', 'tokens', 'rows', 'pages', 'reading_order_sections', 'words', 'sentences', 'blocks', 'vila_entities', 'titles', 'paragraphs', 'authors', 'abstracts', 'keywords', 'sections', 'lists', 'bibliographies', 'equations', 'algorithms', 'figures', 'tables', 'captions', 'headers', 'footers', 'footnotes']

In [4]:
hf_predictor = HfTokenClassificationPredictor(model_name="dslim/bert-base-NER", device="mps")

/Users/sireeshgururaja/miniconda3/envs/papermage/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
hf_predictor.predict(parsed_paper)

0it [00:00, ?it/s]

[Unannotated Entity: {'spans': [[2377, 2378]], 'metadata': {'entity_type': 'MISC'}},
 Unannotated Entity: {'spans': [[2473, 2474]], 'metadata': {'entity_type': 'MISC'}},
 Unannotated Entity: {'spans': [[2831, 2832]], 'metadata': {'entity_type': 'MISC'}},
 Unannotated Entity: {'spans': [[3375, 3376]], 'metadata': {'entity_type': 'MISC'}},
 Unannotated Entity: {'spans': [[3877, 3878]], 'metadata': {'entity_type': 'MISC'}},
 Unannotated Entity: {'spans': [[4868, 4870]], 'metadata': {'entity_type': 'ORG'}},
 Unannotated Entity: {'spans': [[4873, 4890]], 'metadata': {'entity_type': 'ORG'}},
 Unannotated Entity: {'spans': [[5191, 5193]], 'metadata': {'entity_type': 'MISC'}},
 Unannotated Entity: {'spans': [[5195, 5197]], 'metadata': {'entity_type': 'MISC'}},
 Unannotated Entity: {'spans': [[5251, 5252]], 'metadata': {'entity_type': 'MISC'}},
 Unannotated Entity: {'spans': [[5452, 5453]], 'metadata': {'entity_type': 'MISC'}},
 Unannotated Entity: {'spans': [[5515, 5516]], 'metadata': {'entity